In [1]:
import pandas as pd
import numpy as np

# Data Preprocessing

In [2]:
df_train = pd.read_csv('train.csv', parse_dates=['03', '04'])
df_test = pd.read_csv('test.csv', parse_dates=['03', '04'])

print(df_train.shape)
print(df_test.shape)

(55760, 21)
(13940, 20)


In [3]:
# After loading the dataset, map the column names into a human understandable format
column_mapping = {
    'id': 'customer_id',
    '02': 'gender', 
    '03': 'dob', 
    '04': 'lead_creation_date', 
    '05': 'city_code', 
    '06': 'area_code', 
    '07': 'employer_code', 
    '08': 'employer_category1', 
    '09': 'employer_category2', 
    '10': 'monthly_income', 
    '11': 'bank_code', 
    '12': 'bank_acc_type',
    '13': 'marketing_source', 
    '14': 'marketing_category1', 
    '15': 'marketing_category2', 
    '16': 'current_loan_installment', 
    '17': 'requested_loan_amount', 
    '18': 'loan_repayment', 
    '19': 'interest_rate', 
    '20': 'requested_loan_installment', 
    'class': 'loan_approval_status'
}

df_train = df_train.rename(columns=column_mapping)
df_test = df_test.rename(columns=column_mapping)

In [4]:
# Drop the 'requested_loan_installment' column in both train and test datasets
# because this column is redundant with 'current_loan_installment' column based on EDA
df_train = df_train.drop(['requested_loan_installment'],axis=1)
df_test = df_test.drop(['requested_loan_installment'],axis=1)

### Filter out the unwanted rows from training data

##### NaN in 'requested_loan_amount' column
###### Since there is no requested_loan_amount, the loan should not be approved

In [5]:
extracted_nan_train = df_train[df_train[['requested_loan_amount']].isna().any(axis=1)]

# Want to know the customer_id that has loan_approval_status == 1
# In order to remove these data from train dataset
id_extracted_nan_train = extracted_nan_train[extracted_nan_train['loan_approval_status'] == 1]

# Put the customer_id into a list - 148 rows
id_removed_nan_rla_list = id_extracted_nan_train['customer_id'].unique()

# Remove the customer_id from df_train
condition = (df_train['customer_id'].isin(id_removed_nan_rla_list))
df_train_filtered = df_train[~(condition)]

##### dob > lead_creation_date
###### It is impossible to have dob > lead_creation_date, and these data is only 0.0764% of the total training data. Therefore, it is removed.

In [6]:
extracted_year_train = df_train.copy()
extracted_year_train['dob_year'] = pd.DatetimeIndex(extracted_year_train['dob']).year
extracted_year_train['lead_year'] = pd.DatetimeIndex(extracted_year_train['lead_creation_date']).year

extracted_year_train['TF'] = np.where(extracted_year_train['dob_year'] > 2016, 1, 0)

# Want to know the customer_id
# In order to remove these data from train dataset
id_extracted_year_train = extracted_year_train[extracted_year_train['TF'] == 1]

# Put the customer_id into a list
id_removed_year_list = id_extracted_year_train['customer_id'].unique()

# Remove the customer_id from df_train_filtered
condition = (df_train_filtered['customer_id'].isin(id_removed_year_list))
df_train_filtered = df_train_filtered[~(condition)]

### Deal with outliers (numeric columns)

In [7]:
# Identify the quartiles
q1, q3 = np.percentile(df_train_filtered['monthly_income'], [25, 75])
# Calculate the interquartile range
iqr = q3 - q1
# Calculate the lower and upper bounds
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)
# Drop the outliers
clean_data = df_train_filtered[(df_train_filtered['monthly_income'] >= lower_bound) 
                               & (df_train_filtered['monthly_income'] <= upper_bound)]

### Deal with missing values

In [8]:
# Check the number of missing values
clean_data.isnull().sum()

customer_id                     0
gender                          0
dob                             8
lead_creation_date              0
city_code                       0
area_code                     507
employer_code                   0
employer_category1           2487
employer_category2           2668
monthly_income                  0
bank_code                       0
bank_acc_type                6141
marketing_source                0
marketing_category1             0
marketing_category2             0
current_loan_installment       32
requested_loan_amount       19434
loan_repayment              19434
interest_rate               32565
loan_approval_status            0
dtype: int64

#### Numeric columns

In [9]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

clean_data_numeric = clean_data[['current_loan_installment', 'requested_loan_amount', 'loan_repayment', 'interest_rate']]

imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(clean_data_numeric)
column_imputed = pd.DataFrame(imp.transform(clean_data_numeric), columns=['current_loan_installment', 'requested_loan_amount', 'loan_repayment', 'interest_rate'])
column_imputed

,current_loan_installment,requested_loan_amount,loan_repayment,interest_rate
0,0.00,40950.000000,5.000000,19.387415
1,0.00,38531.995062,3.921012,19.347218
2,2063.04,21000.000000,5.000000,15.190000
3,0.00,38531.995062,3.921012,19.347218
4,682.50,51450.000000,5.000000,16.545335
...,...,...,...,...
47640,242.55,26250.000000,4.000000,14.553000
47641,0.00,47250.000000,5.000000,19.600000
47642,840.00,31907.718005,3.808444,17.755997
47643,441.00,35054.249607,3.861914,18.511827


In [10]:
# Drop the original numeric column that have missing values
clean_data = clean_data.drop(['current_loan_installment', 'requested_loan_amount', 'loan_repayment', 'interest_rate'], axis=1)

clean_data2 = pd.concat([clean_data.reset_index(drop=True), column_imputed.reset_index(drop=True)], axis=1)
print(clean_data2.shape)

(47645, 20)


In [11]:
clean_data2.isnull().sum()

customer_id                    0
gender                         0
dob                            8
lead_creation_date             0
city_code                      0
area_code                    507
employer_code                  0
employer_category1          2487
employer_category2          2668
monthly_income                 0
bank_code                      0
bank_acc_type               6141
marketing_source               0
marketing_category1            0
marketing_category2            0
loan_approval_status           0
current_loan_installment       0
requested_loan_amount          0
loan_repayment                 0
interest_rate                  0
dtype: int64

#### Categorical columns

In [ ]:
'''
As for the missing values in categorical columns,
can just fill the missing value with any value that are not in that particular column
'''

In [13]:
clean_data2['area_code'] = clean_data2['area_code'].fillna(4)
clean_data2['employer_category1'] = clean_data2['employer_category1'].fillna(4)
clean_data2['employer_category2'] = clean_data2['employer_category2'].fillna(5)
clean_data2['bank_acc_type'] = clean_data2['bank_acc_type'].fillna(3)

### Train & Test Split

In [14]:
# Remove columns not required
df_train_data = clean_data2.drop(columns = ['customer_id', 'dob', 'lead_creation_date'], axis=1)

# Set the independent and dependent variables
X = df_train_data.drop(['loan_approval_status'], axis=1)
y = df_train_data['loan_approval_status']

# train test splie 80:20
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=37)

# print the shape of X_train and X_test
print(X_train.shape)
print(X_test.shape)

(38116, 16)
(9529, 16)


### Oversampling

In [15]:
from imblearn.over_sampling import RandomOverSampler

# oversample = RandomOverSampler(sampling_strategy='minority')
oversample = RandomOverSampler(sampling_strategy=0.8)
x_over, y_over = oversample.fit_resample(X_train, y_train)

# Modelling - LightGBM

In [16]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

# Creating an object for model and fitting it on training data set 
model = LGBMClassifier()
model.fit(x_over, y_over) # After oversampling (RandomOverSampler)
 
# Predicting the Target variable
pred = model.predict(X_test)
print(pred)
accuracy = model.score(X_test, y_test)
print(accuracy)

[LightGBM] [Info] Number of positive: 30178, number of negative: 37723
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002813 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1634
[LightGBM] [Info] Number of data points in the train set: 67901, number of used features: 16
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444441 -> initscore=-0.223157
[LightGBM] [Info] Start training from score -0.223157
[1 0 0 ... 0 0 0]
0.9247560079756533


### Use train LightGBM to predict test data

In [17]:
# Remove columns not required
df_test_data = df_test.drop(columns = ['customer_id', 'dob', 'lead_creation_date'], axis=1)

## predict test class
x_pred = model.predict(df_test_data)
print(x_pred)

[0 0 0 ... 0 0 0]


In [ ]:
# Creating pandas dataframe from numpy array
submission = pd.DataFrame({'id': np.arange(1,13941), 'class': x_pred})

submission.to_csv('submission.csv', index=False)